In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import numpy as np
from torch.utils.data import Dataset
import pandas as pd
import json
import os
from safetensors.torch import load_file
from transformers import AutoModelForSequenceClassification

os.makedirs("/kaggle/working/best_model", exist_ok=True)

config = {
    "model_type": "deberta",
    "architectures": ["CustomDeBERTa"],
    "num_labels": 3,
    "hidden_size": 768,
    "intermediate_size": 3072,
    "num_attention_heads": 12,
    "num_hidden_layers": 12
}

with open("/kaggle/working/best_model/config.json", "w") as f:
    json.dump(config, f)

max_length = 256
MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
BATCH_SIZE = 16
EPOCHS = 1

class CustomDeBERTa(nn.Module):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.deberta = AutoModel.from_pretrained(model_name)
        self.config = self.deberta.config  # Keep config
        self.config.num_labels = num_labels  # Add number of classes

        hidden_size = self.config.hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.LayerNorm(512),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(256, num_labels)
        )

        # Freeze layers
        for param in self.deberta.parameters():
            param.requires_grad = False
        for layer in self.deberta.encoder.layer[-4:]:
            for param in layer.parameters():
                param.requires_grad = True

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier[-1].out_features),
                          labels.view(-1))

        return {'loss': loss, 'logits': logits}

    def save_pretrained(self, save_directory):
        """Custom method for saving model"""
        os.makedirs(save_directory, exist_ok=True)

        # Save config
        self.config.save_pretrained(save_directory)

        # Save model weights
        if hasattr(self, 'safe_serialization') and self.safe_serialization:
            from safetensors.torch import save_file
            save_file(self.state_dict(), os.path.join(save_directory, "model.safetensors"))
        else:
            torch.save(self.state_dict(), os.path.join(save_directory, "pytorch_model.bin"))

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

def main():
    # Load data
    data = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/train.csv")
    texts = (data['premise'] + " [SEP] " + data['hypothesis']).tolist()
    labels = data['label'].values

    # Tokenizer and datasets
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    dataset = TextDataset(texts, labels, tokenizer, max_length)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

    # Model
    model = CustomDeBERTa(MODEL_NAME, num_labels=3)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        report_to="none"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Training
    trainer.train()

    # Save model
    trainer.save_model("best_model")
    print("Model saved")

    output_dir = "/kaggle/working/best_model"
    os.makedirs(output_dir, exist_ok=True)

    # Save model (with save_pretrained method)
    model.save_pretrained(output_dir)

    # Save tokenizer
    tokenizer.save_pretrained(output_dir)

    print(f"Model and tokenizer saved in {output_dir}")
    print("Directory contents:")
    print(os.listdir(output_dir))

def evaluate_test_data():
    # Load test data
    test_data = pd.read_csv("/kaggle/input/contradictory-my-dear-watson/test.csv")
    test_texts = (test_data['premise'] + " [SEP] " + test_data['hypothesis']).tolist()
    test_ids = test_data['id'].values

    # Model path
    model_path = "/kaggle/working/best_model"

    # Check directory contents
    print("Checking model files:")
    print(os.listdir(model_path))

    # Check required files
    required_files = ['config.json', 'model.safetensors', 'tokenizer_config.json']
    for file in required_files:
        if not os.path.exists(f"{model_path}/{file}"):
            raise FileNotFoundError(f"Required file not found: {file}")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
    except Exception as e:
        print(f"Tokenizer loading error: {e}")
        print("Loading original tokenizer as fallback...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        tokenizer.save_pretrained(model_path)  # Save for future use

    # Load model
    try:
        model = AutoModelForSequenceClassification.from_pretrained(
            model_path,
            trust_remote_code=True  # For custom models
        )
    except Exception as e:
        print(f"Model loading error: {str(e)}")
        raise

    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Batch processing for memory efficiency
    predictions = []
    for i in range(0, len(test_texts), BATCH_SIZE):
        batch_texts = test_texts[i:i+BATCH_SIZE]

        # Tokenization with max length handling
        inputs = tokenizer(
            batch_texts,
            truncation=True,
            padding='longest',  # Auto-padding to longest sequence in batch
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        # Prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs[0]
            batch_preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(batch_preds)

    if len(predictions) != len(test_ids):
        raise ValueError(f"Predictions count ({len(predictions)}) doesn't match test examples ({len(test_ids)})")

    # Save results
    submission_df = pd.DataFrame({
        'id': test_ids,
        'prediction': predictions
    })

    # Format validation
    if not all(submission_df['prediction'].between(0, 2)):  # For 3 classes (0,1,2)
        raise ValueError("Predictions must be in range 0-2")

    submission_df.to_csv("/kaggle/working/submission.csv", index=False)

    # Additional validation
    print("Checking saved files:")
    print(os.listdir("/kaggle/working/"))

    print("\nFirst 5 rows of submission.csv:")
    print(submission_df.head())

if __name__ == "__main__":
    main()
    evaluate_test_data()